<a href="https://colab.research.google.com/github/mhpbreugem/BBP/blob/main/Hellwig_1980_OLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
using LinearAlgebra, BenchmarkTools, Symbolics, NLsolve


Hellwig (1980) OLG
==========================================================================

## 📘 Description

This notebook simulates a **dynamic OLG version** of the rational expectations model from **Hellwig (1980)**.

In contrast to the static original, this version incorporates a **future price** $P_1$ directly into the agent’s payoff function:

$$
\text{Payoff} = W_0 + X_0 \cdot (D_1 + P_1 - P_0)
$$

Agents form beliefs about the payoff-relevant latent state using:

- A **private signal** $S_1 = D_1 + \varepsilon_1$
- A **public price** signal $P_0 = c_1 D_1 + c_z Z_0$

They use **Bayesian updating** to derive posterior beliefs, compute optimal demand using **CARA utility**, and prices adjust to ensure **market clearing**.

The function `φSYS0` computes the **fixed point** in price coefficients $(c_1, c_z)$ such that individual behavior and equilibrium pricing are consistent.



---

Step 1: Conjectured Price Function
----------------------------------

We start by postulating that prices at time 0 are linear in the fundamentals:

$$P_0 = c_1 D_1 + c_z Z_0 $$

We omit lagged prices ($P_{-1}$) because the fixed point equilibrium implies $c_p = 0$.

The latent vector of economic fundamentals is:

$$F = [D_1, D_2, Z_0, Z_1, \varepsilon_1]$$

---

Step 2: Signal Structure
------------------------

Agents observe two signals:

- A private signal:
$$S_1 = D_1 + \varepsilon_1 \quad \Rightarrow \quad \Omega_{S_1} = [1, 0, 0, 0, 1]$$

- A public signal (price):
$$P_0 = c_1 D_1 + c_z Z_0 \quad \Rightarrow \quad \Omega_{P_0} = [c_1, 0, c_z, 0, 0]$$

We construct the signal matrix:

$$S = \text{hcat}(\Omega_{S_1}, \Omega_{P_0}) \in \mathbb{R}^{5 \times 2}$$

---

Step 3: Bayesian Updating
-------------------------

We compute posterior beliefs based on observed signals. Define:

- $\Sigma_F$: covariance of the latent variables
- $\Sigma_{YY} = S^\top \Sigma_F S$: covariance of the signals
- $\Sigma_{FY} = \Sigma_F S$: cross-covariance

The posterior mean mapping is:

$$B = \Sigma_{FY} \Sigma_{YY}^{-1} \quad \Rightarrow \quad \mathbb{E}[F \mid S] = B S$$

The posterior covariance of the latent variables is:

$$\Sigma_{\text{post}} = \Sigma_F - \Sigma_{FY} \Sigma_{YY}^{-1} \Sigma_{FY}^\top$$

---

Step 4: Individual Demand
-------------------------

Agents maximize expected utility with CARA preferences. Their consumption payoff is:

$$CS = W_0 + X_0 (D_1 + P_1 - P_0) \quad \Rightarrow \quad L := D_1 + P_1 - P_0$$

Using the price conjecture at time 1:

$$P_1 = c_1 D_2 + c_z Z_1$$

we substitute:

$$L = D_1 + c_1 D_2 + c_z Z_1 - P_0$$

This is a linear function of $F$, with loading:

 $$\lambda = [1, c_1, 0, c_z, 0]$$

Posterior mean of $L$ given signals:

$$\mathbb{E}[L \mid S] = \lambda^\top B S + (c_p - 1) P_0$$

Let:

$$EL = \lambda^\top B = [EL_1, EL_2]$$

and define posterior variance:

$$\sigma_L^2 = \lambda^\top \Sigma_{\text{post}} \lambda$$

Then the agent's optimal demand is:

$$X_0 = \frac{EL_1 S_1 + (EL_2 - 1) P_0}{\gamma \sigma_L^2} = z_1 S_1 + z_2 P_0$$

with:

$$z_1 = \frac{EL_1}{\gamma \sigma_L^2}, \quad z_2 = \frac{EL_2 - 1}{\gamma \sigma_L^2}$$

---

Step 5: Market Clearing and Fixed Point
---------------------------------------

In equilibrium, aggregate demand equals the exogenous noise supply:

$$Z_0 = z_1 D_1 + z_2 P_0$$

Solve for $P_0$:

$$P_0 = \frac{1}{z_2} Z_0 - \frac{z_1}{z_2} D_1$$

Compare this to the original conjecture:

$$P_0 = c_1 D_1 + c_z Z_0$$

Matching coefficients implies the fixed point conditions:

$$c_1 = -\frac{z_1}{z_2}, \quad c_z = \frac{1}{z_2}$$

Therefore, the residuals returned by the function are:

$$\text{residual}_1 = c_1 + \frac{z_1}{z_2}, \quad \text{residual}_2 = c_z - \frac{1}{z_2}$$



In [2]:
function φSYS0(c, ΣF, γ)
    # Inputs:
    # c: Vector of conjectured price coefficients [c1, cz, cp]
    # ΣF: Covariance matrix of latent variables F = [D1, D2, Z0, Z1, ε_1]
    # γ: Risk aversion parameter

    # --- Step 1: Conjectured Price Function Coefficients ---
    c1, cz = c

    # --- Step 2: Define Signal Matrix ---
    ΩS1 = [1.0, 0.0, 0.0, 0.0, 1.0]     # S₁ = D₁ + ε₁
    ΩP0 = [c1 , 0.0, cz , 0.0, 0.0]     # P₀ = c₁ D₁ + c_z Z₀

    S = hcat(ΩS1, ΩP0)                  # 5×2 signal matrix

    # --- Step 3: Bayesian Updating ---
    ΣYY = transpose(S) * ΣF * S
    ΣFY = ΣF * S
    B = ΣFY * inv(ΣYY)                  # 5×2 posterior mean coefficients
    Σ_post = ΣF - ΣFY * inv(ΣYY) * transpose(ΣFY)

    # --- Step 4: Compute Optimal Demand Coefficients ---
   #λ = [1.0, 0.0, 0.0, 0.0, 0.0]        (using this λ removes P1 from the payoff and implies Hellwig 1980)
    λ = [1.0, c1, 0.0, cz, 0.0]         # L = D₁ + P₁ - P₀ (L = λ'F -P₀)
    σL2 = dot(λ, Σ_post * λ)
    EL = transpose(λ) * B               # E[L | signals] = EL₁ * S₁ + EL₂ * P₀

    z1 =  EL[1]      / (γ * σL2)
    z2 = (EL[2] - 1) / (γ * σL2)

    # --- Step 5: Verify Conjecture by Fixed Point Conditions ---
    c1_new = -z1 / z2
    cz_new = 1 / z2

    return [c1 - c1_new, cz - cz_new]
end


φSYS0 (generic function with 1 method)

In [3]:
using Optim

function solve_optim()
    ΣF = diagm([1,1,1,1,0.1])
    γ = 1
    φ_wrapped(c) = sum(abs2, φSYS0(c, ΣF, γ))
    result = optimize(φ_wrapped, [1, -1.12])
    return Optim.minimizer(result)
end

solve_optim()

2-element Vector{Float64}:
  0.5632110545571346
 -0.6727938454264809